# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,41.88,89,100,17.94,TF,1699827893
1,1,malvan,16.0667,73.4667,79.50,72,100,4.72,IN,1699828220
2,2,isafjordur,66.0755,-23.1240,35.56,82,100,3.91,IS,1699827896
3,3,hithadhoo,-0.6000,73.0833,80.89,80,90,5.59,MV,1699827877
4,4,bilibino,68.0546,166.4372,8.91,95,100,4.65,RU,1699827878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    alpha = 0.7,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.dropna()

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# A maximum temperature below 85 degrees but above 75 degrees
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 85) & (ideal_weather_df["Max Temp"] > 75)]

# Humidity below 40%
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Humidity"] < 40]

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,ghanzi,-21.5667,21.7833,82.08,29,47,7.81,BW,1699828223
33,33,guerrero negro,27.9769,-114.0611,78.82,24,0,8.79,MX,1699827873
76,76,taoudenni,22.6783,-3.9836,81.97,15,0,13.67,ML,1699827910
149,149,tazacorte,28.6290,-17.9293,75.15,39,100,8.05,ES,1699827901
183,183,newman,37.3138,-121.0208,76.28,23,78,6.20,US,1699827717
189,189,laguna,38.4210,-121.4238,76.17,32,0,10.36,US,1699827710
280,280,tchintabaraden,15.8969,5.7985,77.50,14,0,10.54,NE,1699828253
286,286,mangrol,21.1167,70.1167,77.07,31,96,9.73,IN,1699828253
299,299,faya-largeau,17.9257,19.1043,80.55,19,6,13.71,TD,1699827904
351,351,alaghsas,17.0187,8.0168,76.66,14,0,12.82,NE,1699828008


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,ghanzi,BW,-21.5667,21.7833,29,
33,guerrero negro,MX,27.9769,-114.0611,24,
76,taoudenni,ML,22.6783,-3.9836,15,
149,tazacorte,ES,28.6290,-17.9293,39,
183,newman,US,37.3138,-121.0208,23,
189,laguna,US,38.4210,-121.4238,32,
280,tchintabaraden,NE,15.8969,5.7985,14,
286,mangrol,IN,21.1167,70.1167,31,
299,faya-largeau,TD,17.9257,19.1043,19,
351,alaghsas,NE,17.0187,8.0168,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel",
    "limit" : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghanzi - nearest hotel: Ghanzi farmhouse
guerrero negro - nearest hotel: Plaza sal paraiso
taoudenni - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
newman - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tchintabaraden - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
alaghsas - nearest hotel: Hôtel Zagado
maun - nearest hotel: Center Lodge Conference Center
san - nearest hotel: La Roche
paraguacu - nearest hotel: No hotel found
menaka - nearest hotel: No hotel found
pindiga - nearest hotel: No hotel found
mayahi - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
maamba - nearest hotel: No hotel found
nguigmi - nearest hotel: Guest PAM
markala - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast


,City,Country,Lat,Lng,Humidity,Hotel Name
13,ghanzi,BW,-21.5667,21.7833,29,Ghanzi farmhouse
33,guerrero negro,MX,27.9769,-114.0611,24,Plaza sal paraiso
76,taoudenni,ML,22.6783,-3.9836,15,No hotel found
149,tazacorte,ES,28.6290,-17.9293,39,App Leyma
183,newman,US,37.3138,-121.0208,23,No hotel found
189,laguna,US,38.4210,-121.4238,32,Holiday Inn Express & Suites
280,tchintabaraden,NE,15.8969,5.7985,14,No hotel found
286,mangrol,IN,21.1167,70.1167,31,No hotel found
299,faya-largeau,TD,17.9257,19.1043,19,No hotel found
351,alaghsas,NE,17.0187,8.0168,14,Hôtel Zagado


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    alpha = 0.7,
    color = "City",
    tiles = "OSM",
    hover_cols=["Country", "Hotel Name"],
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)